# Portfolio Optimization Project

### CS/ECE/ISyE 524 — Introduction to Optimization — Fall 2024

### Project Members: Pawin Linmaneechote, Vincent Philavong, Jiajie Yao

## Table of Contents
1. Introduction
2. Mathematical Model
3. Solution
4. Results and Discussion
5. Conclusion
6. Appendix


# 1. Introduction


# 2. Mathematical Model

In [10]:
using CSV, DataFrames, Dates

directory_path = "data_files/"

result_df = DataFrame(StockName = String[], OpenStock = Float64[], 
                      CloseStock = Float64[], EquityPercent = Float64[])

stock_sector = CSV.read("Stock Sheet.csv", DataFrame)
rename!(stock_sector, Symbol("Ticker") => :StockName) 

for file in readdir(directory_path)
    if endswith(file, ".csv")
        file_path = joinpath(directory_path, file)
        stock_df = CSV.read(file_path, DataFrame)
        
        stock_df.Date = Date.(string.(stock_df.Date), "yyyy-mm-dd")
        sort!(stock_df, :Date)  
        
        stock_name = splitext(basename(file))[1]
        
        open_price = stock_df.Open[1]
        close_price = stock_df.Close[end]

        equity_percent = ((close_price - open_price) / open_price) * 100

        push!(result_df, (stock_name, open_price, close_price, equity_percent))
    end
end

final_df = leftjoin(result_df, stock_sector[:, [:StockName, :StockSector]], on = :StockName)

select!(final_df, [:StockName, :OpenStock, :CloseStock, :EquityPercent, :StockSector])

println(final_df)

25×5 DataFrame
 Row │ StockName  OpenStock  CloseStock  EquityPercent  StockSector            
     │ String     Float64    Float64     Float64        String31?              
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ AAPL         64.1925      223.45       248.094   Technology
   2 │ UBER         26.06         74.15       184.536   Technology
   3 │ NVDA          5.216       139.91      2582.32    Technology
   4 │ MSFT        144.37        411.46       185.004   Technology
   5 │ AMD          36.06        141.66       292.845   Technology
   6 │ DFS          83.24        151.85        82.4243  Financial
   7 │ JPM         128.69        221.49        72.1113  Financial
   8 │ SYF          36.34         56.82        56.3566  Financial
   9 │ AXP         118.89        275.94       132.097   Financial
  10 │ GS          218.01        526.96       141.714   Financial
  11 │ PFE          35.6831       27.99       -21.5595  Health Care
  12 │ JNJ  

In [69]:
using CSV, DataFrames, Dates, Statistics

directory_path = "data_files/"

# Initialize an empty DataFrame
result_df = DataFrame(StockName = String[], OpenStock = Float64[], CloseStock = Float64[],
                      EquityPercent = Float64[], Date = Date[])

# Read stock sector data
stock_sector = CSV.read("Stock Sheet.csv", DataFrame)
rename!(stock_sector, Symbol("Ticker") => :StockName)

# Process each file in the directory
for file in readdir(directory_path)
    if endswith(file, ".csv")
        file_path = joinpath(directory_path, file)
        stock_df = CSV.read(file_path, DataFrame)
        
        stock_df.Date = Date.(string.(stock_df.Date), "yyyy-mm-dd")
        sort!(stock_df, :Date)  

        stock_name = splitext(basename(file))[1]
        
        # Calculate Open and Close prices
        open_price = stock_df.Open[1]
        close_price = stock_df.Close[end]

        # Calculate daily EquityPercent
        stock_df.EquityPercent = ((stock_df.Close .- stock_df.Open) ./ stock_df.Open) * 100

        # Append OpenStock, CloseStock, and EquityPercent to result_df
        for row in eachrow(stock_df)
            push!(result_df, (stock_name, open_price, close_price, row.EquityPercent, row.Date))
        end
    end
end

# Group by StockName to compute EquityStdDev
grouped = groupby(result_df, :StockName)
summary_df = combine(grouped, 
    :OpenStock => first,              # First OpenStock for each stock
    :CloseStock => last,              # Last CloseStock for each stock
    :EquityPercent => std => :EquityStdDev,  # Standard deviation of EquityPercent
    :EquityPercent => first => :EquityPercent  # Use first EquityPercent value
)

# Join with StockSector to include the sector
final_df = leftjoin(summary_df, stock_sector[:, [:StockName, :StockSector]], on = :StockName)

rename!(final_df, Dict(
    :OpenStock_first => :OpenStock,
    :CloseStock_last => :CloseStock
))

final_df

select!(final_df, [:StockName, :OpenStock, :CloseStock, :EquityPercent, :EquityStdDev, :StockSector])

println("Final DataFrame:")
final_df

Row,StockName,OpenStock,CloseStock,EquityPercent,EquityStdDev,StockSector
,String,Float64,Float64,Float64,Float64,String31?
1,AAPL,64.1925,223.45,0.183045,1.53954,Technology
2,UBER,26.06,74.15,3.37683,2.72985,Technology
3,NVDA,5.216,139.91,-0.484087,2.67358,Technology
4,MSFT,144.37,411.46,-0.214724,1.44934,Technology
5,AMD,36.06,141.66,-0.360513,2.73579,Technology
6,DFS,83.24,151.85,-0.0720783,2.18694,Financial
7,JPM,128.69,221.49,0.474008,1.37905,Financial
8,SYF,36.34,56.82,1.32086,2.15037,Financial
9,AXP,118.89,275.94,0.672893,1.67668,Financial


In [75]:
equity_pct = final_df.EquityPercent

# Compute the covariance matrix
n = length(equity_pct)
cov_matrix = zeros(n, n)  # Initialize a covariance matrix

for i in 1:n
    for j in 1:n
        cov_matrix[i, j] = (equity_pct[i] - mean(equity_pct)) * (equity_pct[j] - mean(equity_pct)) / (n - 1)
    end
end

println("Covariance Matrix (Without Correlations):")
cov_matrix

25×25 Matrix{Float64}:
  0.000157718   0.00834503  -0.00155248   …  -0.00225916    0.000959636
  0.00834503    0.441543    -0.0821434       -0.119534      0.0507752
 -0.00155248   -0.0821434    0.0152817        0.0222378    -0.00944607
 -0.000861967  -0.0456075    0.00848468       0.0123468    -0.00524463
 -0.0012357    -0.065382     0.0121635        0.0177001    -0.0075186
 -0.000496293  -0.0262593    0.00488521   …   0.0071089    -0.00301969
  0.000903607   0.0478107   -0.00889455      -0.0129433     0.00549798
  0.00307452    0.162676    -0.0302637       -0.0440394     0.0187069
  0.00141345    0.0747871   -0.0139132       -0.0202463     0.00860013
  0.000170591   0.00902612  -0.00167919      -0.00244354    0.00103796
  ⋮                                       ⋱                
  0.000153669   0.00813077  -0.00151262      -0.00220115    0.000934997
  4.42842e-5    0.00234312  -0.000435906     -0.000634326   0.000269446
  0.000505296   0.0267357   -0.00497382      -0.00723785    0.003

In [ ]:
function extract_adj_close(filenames::Vector{String})
    
    adj_close_data = []

    for file in filenames
        df = CSV.read(file, DataFrame)
        push!(adj_close_data, df[!, "Adj Close"])
    end

    return hcat(adj_close_data...)  
    
end
returns_matrix = extract_adj_close(data_files)
println("Returns Matrix:\n", returns_matrix)

UndefVarError: UndefVarError: `CloseStock` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [21]:
function compute_covariance(directory_path::Matrix{Float64})
    
    cov_matrix = cov(returns)
    return cov_matrix
    
end
cov_matrix_data = compute_covariance(returns_matrix)
println("Covariance matrix of three different assets:\n", cov_matrix_data)

UndefVarError: UndefVarError: `returns_matrix` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
using JuMP, Ipopt, PyPlot, LinearAlgebra

# Given parameters
#=
μ = [0.1605, 0.1194, 0.0063, 0.1608, 0.1914]

Σ = [0.478772729 -0.094105897 0.011735631 -0.086955984 -0.090339195; 
    -0.094105897 0.404044905 0.074752991 0.235695145 0.342544199;
    0.011735631 0.074752991 1.1 -0.682850019 -0.076567893;
    -0.086955984 0.235695145 -0.682850019 0.900054574 0.407742435;
    -0.090339195 0.342544199 -0.076567893 0.407742435 0.530447761]
=#

μ = final_df.EquityPercent
Σ = cov_matrix

R_min = 0.1  # Single R_min value
λ = 1  # Fixed λ

# Create model
m = Model(Ipopt.Optimizer)
@variable(m, x[1:25] >= 0)
@constraint(m, sum(x) == 1)
@constraint(m, μ'*x >= R_min)
@objective(m, Min, -μ'*x + λ * (x'*Σ*x))

# Solve
optimize!(m)

# Extract results
status = termination_status(m)
display(status)

status = termination_status(m)
if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
    println("The model is feasible.")
    figure(figsize=(10, 6))
    bar(1:5, portfolio_weights, color="blue", alpha=0.7)
    xticks(1:5, ["Asset $i" for i in 1:5])
    xlabel("Assets")
    ylabel("Portfolio Weight")
    title("Optimal Portfolio Weights for R_min = $R_min")
    grid(true)
    show()
    display(gcf())

    # Print portfolio metrics
    display("Portfolio Return: $portfolio_return")
    display("Portfolio Risk (Variance): $portfolio_risk")
    
else
    println("The model is not feasible. Status: $status")
end

In [ ]:
function sensitivity_analysis(Σ::Matrix{Float64}, perturbation::Float64)
    n_assets = size(Σ, 1)

    # Solve the original problem
    original_weights, original_risk = min_portfolio_risk(cov_matrix)

    # Perturb the covariance matrix
    perturbed_cov_matrix = Σ .+ perturbation * rand(n_assets, n_assets)

    # Solve the perturbed problem
    new_weights, new_risk = min_portfolio_risk(perturbed_cov_matrix)

    return original_weights, original_risk, new_weights, new_risk
end
perturbation = 0.1  
original_weights, original_risk, new_weights, new_risk = sensitivity_analysis(cov_matrix_data, perturbation)

println("Original weights: ", original_weights)
println("Original risk: ", original_risk)
println("New weights after perturbation: ", new_weights)
println("New risk after perturbation: ", new_risk)


# 3. Solutions

# 4. Results and Discussion

# 5. Conclusion

# 6. Appendix